<a href="https://colab.research.google.com/github/srdg/unarchived_ben_tess/blob/main/Bengali_Typeset_With_Tesseract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation
Mount tesseract and set up the environment by installing  necessary libraries and packages

In [18]:
!sudo apt install -q tesseract-ocr

Reading package lists...
Building dependency tree...
Reading state information...
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.


In [19]:
!sudo apt install libtesseract-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libtesseract-dev is already the newest version (4.00~git2288-10f4998a-2).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.


In [20]:
!which tesseract

/usr/bin/tesseract


In [21]:
!sudo apt-get install tesseract-ocr-ben

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr-ben is already the newest version (4.00~git24-0e00fe6-1.2).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.


In [22]:
!ls /usr/share/tesseract-ocr/4.00/tessdata

ben.traineddata  eng.traineddata  pdf.ttf
configs		 osd.traineddata  tessconfigs


# Downloading data

In [25]:
!rm -rf ./sample_data # remove default sample data
!rm -rf 010_LetterPress-Data line_hocr.sh # delete old versions of data
!wget -q https://api.github.com/repos/srdg/unarchived_ben_tess/zipball  
!unzip -q zipball && unzip -q ./srdg-*/010_LetterPress.zip
!mv ./srdg-*/line_hocr.sh . #unzip data and take the shell script
!chmod +xxx ./line_hocr.sh # make executable
!rm -rf zipball srdg-* # delete redundant archives

In [26]:
!wget https://github.com/ocropus/hocr-tools/archive/master.zip && \
unzip -q ./master.zip
!rm -rf ./master.zip # get hocr-tools

--2020-12-12 09:51:57--  https://github.com/ocropus/hocr-tools/archive/master.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/ocropus/hocr-tools/zip/master [following]
--2020-12-12 09:51:57--  https://codeload.github.com/ocropus/hocr-tools/zip/master
Resolving codeload.github.com (codeload.github.com)... 140.82.114.9
Connecting to codeload.github.com (codeload.github.com)|140.82.114.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [ <=>                ]   1.46M  --.-KB/s    in 0.1s    

2020-12-12 09:51:57 (12.5 MB/s) - ‘master.zip’ saved [1532153]

replace hocr-tools-master/.gitignore? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# Preprocessing data

In [14]:
# Run shell script to split lines
!bash line_hocr.sh&>./preprocessing.log

In [15]:
# Move the split images into separate folder to correct ground truth
!mkdir -p ./data/ben-ground-truth
!mv ./010_LetterPress-Data/*.exp0.* ./data/ben-ground-truth/

In [17]:
!zip -r -q data.zip ./data # zip for downloading

In [27]:
# After correcting ground truth, upload data again
from google.colab import files
files.upload()

Saving data.zip to data.zip
caution: filename not matched:  .


In [28]:
# Delete old directory with unclean data and unzip archive
!rm -rf data
!unzip -q data.zip
!rm -rf data.zip # remove redundant archive file

# Training

In [29]:
!rm -rf tesstrain-master # remove old downloads for session continuity
!wget https://github.com/tesseract-ocr/tesstrain/archive/master.zip && \
unzip ./master.zip # unzip training diretory
!rm -rf ./master.zip # remove redundant archive

--2020-12-12 09:57:34--  https://github.com/tesseract-ocr/tesstrain/archive/master.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/tesseract-ocr/tesstrain/zip/master [following]
--2020-12-12 09:57:34--  https://codeload.github.com/tesseract-ocr/tesstrain/zip/master
Resolving codeload.github.com (codeload.github.com)... 140.82.114.10
Connecting to codeload.github.com (codeload.github.com)|140.82.114.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [  <=>               ]   5.19M  15.8MB/s    in 0.3s    

2020-12-12 09:57:35 (15.8 MB/s) - ‘master.zip’ saved [5445507]

Archive:  ./master.zip
c72c7d1225d030ca9db5990b2a581f82f3a4e5b6
   creating: tesstrain-master/
   creating: tesstrain-master/.github/
  inflating: tesstrain-

In [30]:
!mv ./data ./tesstrain-master/

In [37]:
%cd ./tesstrain-master/
import os
print(os.getcwd())

/content/tesstrain-master
/content/tesstrain-master


In [ ]:
# temporary - fix stupid mistake of not renaming .txt files to .gt.txt
%cd ./data/ben-ground-truth/
!rename 's/.exp0.txt$/.exp0.gt.txt/' *.exp0.txt
%cd ../..
print(os.getcwd())

In [43]:
# Cleanup
!rm -rf ./data/ben 
!rm -rf radical-stroke.txt
!rm -rf ./data/ben-ground-truth/*.box ./data/ben-ground-truth/*.lstmf

In [44]:
!make training MODEL_NAME=ben

make: Warning: File 'data/ben-ground-truth/20-012.exp0.gt.txt' has modification time 18194 s in the future
find data/ben-ground-truth -name '*.gt.txt' | xargs cat | sort | uniq > "data/ben/all-gt"
unicharset_extractor --output_unicharset "data/ben/unicharset" --norm_mode 2 "data/ben/all-gt"
Bad box coordinates in boxfile string! তিনিই ছিলেন সর্বাপেক্ষা প্রাচীন ও বিজ্ঞ, এজন্য সকলেই তাঁকে শ্রদ্ধাক'রে দেখ। সেই সেদিন বললে না ?বট গাছের ছায়ায় সে বসেছে। সন্ধা তখনও নামেনি, ঝিরঝিরে বাতাসেসুরদাসের কথা কেমন অসংলগ্ন বোধ হতে লাগল, তাঁর মুখের দিকে চেয়েঘটপূর্ণ জল দেবীর গায়ে ছিটিয়ে দিও।সুনন্দা বললে—আমি কি একা ছিলাম ? দুপুর বেলায় আমি একাতাঁর, না জানি কত সুন্দর তাঁর মুখশ্ৰী! আচাৰ্য বসুব্রত বলেন বটে......দেবী বললেন—না। এক সন্ন্যাসী আমায় এখানে সঙ্গে করে এনেছেন,জ্যোৎস্না পথ ঘাট ধুইয়ে দিচ্ছিল। দুর মাঠের গাছপালা জোৎস্নায় ঝাপসাতার অবাধ্য মন যে এই সব পরিপূর্ণ জোৎস্নারাত্রে মহাকোটঠি বিহারেরসেদিন ঘুরতে ঘুরতে অবসন্ন অবস্থায় উরুবিল্ব গ্রামের প্রান্তে একটা বড়বলছ ? আমার দেশ কোথায় জানিনে। আমি নাকি বিদিশার

UnicodeDecodeError: ignored